#### Author: Piaotian Jin, Elizabeth Rogers
#### Date: December 10, 2017
#### Description: This is a ArcGIS tool to simulate urban expansion based on CA model.

In [ ]:
import arcpy
from arcpy.sa import *
import os

In [ ]:
# Set working environment
arcpy.env.overwriteOutput = True

In [ ]:
# Recieves input factor layers form user and corresponding weights
urban_nonurban = arcpy.GetParameterAsText(0)
weight_neigh = arcpy.GetParameterAsText(1)

factor1 = arcpy.GetParameterAsText(2)
weight1 = arcpy.GetParameterAsText(3)

factor2 = arcpy.GetParameterAsText(4)
weight2 = arcpy.GetParameterAsText(5)

factor3 = arcpy.GetParameterAsText(6)
weight3 = arcpy.GetParameterAsText(7)

factor4 = arcpy.GetParameterAsText(8)
weight4 = arcpy.GetParameterAsText(9)

factor5 = arcpy.GetParameterAsText(10)
weight5 = arcpy.GetParameterAsText(11)

constraint = arcpy.GetParameterAsText(12)

iteration = arcpy.GetParameterAsText(13)
output = arcpy.GetParameterAsText(14)

# initialize variables
city = urban_nonurban


In [ ]:
# run CA model
i = 0
while i < int(iteration):
    neigh = FocalStatistics(city, NbrRectangle(3, 3, "CELL"), "MEAN", "DATA")
    nonurban1 = Reclassify(city, "Value", RemapValue([[1, 0], [0, 1]]))

    # conditional if statement if the user inputs a constraint layer
    if constraint:
        # conditional statements that check for optional factors and executes the CA model
        # according to the number of factors that are present
        if factor5:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            times4 = Times(factor3, float(weight3))
            times5 = Times(factor4, float(weight4))
            times6 = Times(factor5, float(weight5))
            cell_value = times1 + times2 + times3 + times4 + times5 + times6
            urban_value = cell_value * nonurban1 * constraint
        elif factor4:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            times4 = Times(factor3, float(weight3))
            times5 = Times(factor4, float(weight4))
            cell_value = times1 + times2 + times3 + times4 + times5
            urban_value = cell_value * nonurban1 * constraint
        elif factor3:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            times4 = Times(factor3, float(weight3))
            cell_value = times1 + times2 + times3 + times4
            urban_value = cell_value * nonurban1 * constraint
        else:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            cell_value = times1 + times2 + times3
            urban_value = cell_value * nonurban1 * constraint

    # conditional statements that check for optional factors and executes the CA model
    # according to the number of factors that are present if NO constraint is used
    else:
        if factor5:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            times4 = Times(factor3, float(weight3))
            times5 = Times(factor4, float(weight4))
            times6 = Times(factor5, float(weight5))
            cell_value = times1 + times2 + times3 + times4 + times5 + times6
            urban_value = cell_value * nonurban1
        elif factor4:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            times4 = Times(factor3, float(weight3))
            times5 = Times(factor4, float(weight4))
            cell_value = times1 + times2 + times3 + times4 + times5
            urban_value = cell_value * nonurban1
        elif factor3:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            times4 = Times(factor3, float(weight3))
            cell_value = times1 + times2 + times3 + times4
            urban_value = cell_value * nonurban1
        else:
            times1 = Times(neigh, float(weight_neigh))
            times2 = Times(factor1, float(weight1))
            times3 = Times(factor2, float(weight2))
            cell_value = times1 + times2 + times3
            urban_value = cell_value * nonurban1

    new_urban = Reclassify(urban_value, "Value", RemapRange([[0, 0.5, 0], [0.5, 1, 1]]), 0)

    # adds new urban areas to existing urban areas and redefines "city" variable
    # this is then put back through the while loop the number of iterations dictated by user
    city = new_urban + city
    i = i + 1

In [ ]:
# creates final output layer with new urban areas, existing urban areas, and non-urban areas
new_cell = city - urban_nonurban
final = city + new_cell
final.save(output + "\\result.tif")

In [ ]:
# write information file
report = open(output + "\\report.txt", "w")

# create a dictionary to store the number of pixels of each values
count = {}
count_cursor = arcpy.da.SearchCursor(output + "\\result.tif", ["VALUE", "COUNT"])
for row in count_cursor:
    count[count_cursor[0]] = count_cursor[1]

report.write(
    "This is a report about the areas and the number of cells that changed after %d iterations.\n\n" % int(iteration))
report.write("Rows are before iterations. Columns are after iterations.\n\n")

# write the number of cells
report.write("The number of cells:\n\n")
report.write("                    Urban       Non-urban\n")
report.write("Urban      %14d%16d\n" % (count[1], 0))
report.write("Non-urban  %14d%16d\n\n\n" % (count[2], count[0]))

# write the area of change in square units
spatial_ref = arcpy.Describe(urban_nonurban).spatialReference
units = spatial_ref.linearUnitName
X = arcpy.GetRasterProperties_management(urban_nonurban, "CELLSIZEX")
Y = arcpy.GetRasterProperties_management(urban_nonurban, "CELLSIZEY")
cell_area = float(X.getOutput(0)) * float(Y.getOutput(0))
report.write("Areas(Square %ss):\n\n" % units)
report.write("                    Urban       Non-urban\n")
report.write("Urban      %14d%16d\n" % (count[1] * cell_area, 0))
report.write("Non-urban  %14d%16d\n\n\n" % (count[2] * cell_area, count[0] * cell_area))
report.close()

In [ ]:
# make map and exported as PDF file
try:
    mxd = arcpy.mapping.MapDocument(output + "\\template.mxd")
    df1 = arcpy.mapping.ListDataFrames(mxd, "Data Frame 1")[0]
    df2 = arcpy.mapping.ListDataFrames(mxd, "Data Frame 2")[0]
    legend1 = arcpy.mapping.ListLayoutElements(mxd, "LEGEND_ELEMENT", "Legend")[0]
    legend2 = arcpy.mapping.ListLayoutElements(mxd, "LEGEND_ELEMENT", "Legend")[1]
    legend1.autoAdd = True
    legend2.autoAdd = True
    addLayer1 = arcpy.mapping.Layer(urban_nonurban)
    addLayer2 = arcpy.mapping.Layer(output + "\\result.tif")
    arcpy.mapping.AddLayer(df1, addLayer1, "BOTTOM")
    arcpy.mapping.AddLayer(df2, addLayer2, "BOTTOM")
    template_lyr = arcpy.mapping.Layer(output + "\\template.lyr")
    lyr1_name = os.path.basename(urban_nonurban)
    lyr1 = arcpy.mapping.ListLayers(mxd, lyr1_name, df1)[0]
    lyr2 = arcpy.mapping.ListLayers(mxd, "result.tif", df2)[0]

    # change symbology type to "RASTER_CLASSIFIED"
    arcpy.mapping.UpdateLayer(df1, lyr1, template_lyr, True)
    arcpy.mapping.UpdateLayer(df2, lyr2, template_lyr, True)

    # change the label
    if lyr1.symbologyType == "RASTER_CLASSIFIED":
        lyr1.symbology.classBreakValues = [0, 0.9, 2]
        lyr1.symbology.classBreakLabels = ["Non-urban", "Urban"]
    else:
        arcpy.AddMessage("There was an error when making the map. Please check the template layer.")
    if lyr2.symbologyType == "RASTER_CLASSIFIED":
        lyr2.symbology.classBreakValues = [0, 0.9, 1.9, 3]
        lyr2.symbology.classBreakLabels = ["Non-urban", "Existing Urban", "New Urban"]
    else:
        arcpy.AddMessage("There was an error when making the map. Please check the template layer.")

    arcpy.RefreshActiveView()
    arcpy.RefreshTOC()

    arcpy.mapping.ExportToPDF(mxd, output + "\\resultMap.pdf")
    del mxd

except:
    arcpy.AddMessage("There was an error when making the map. Please check the completeness of default files.")